#### 1.) 학습모델 Reconstruction error 기반 전체 Prediction result
- IsolationForest ( 데이터 크기 소)
- AutoEncoder ( 데이터 크기 중)
- Memory Augmented AutoEncoder ( 데이터 크기 대)

In [1]:
import os
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
from tqdm import tqdm

adjusted_base_path = "임시라벨링_결과"
folders = ["maker1","maker2","maker"]
for folder in folders:
    folder_path = os.path.join(adjusted_base_path, folder)
    file_list = [f for f in os.listdir(folder_path) if f.endswith(".csv")] # _labeled.csv
    dfs = []
    print(f"\n📂 Concatenating folder: {folder}")
    for file_name in tqdm(file_list, desc=f"Reading {folder}", ncols=100):
        file_path = os.path.join(folder_path, file_name)
        df = pd.read_csv(file_path, encoding='cp949')
        df["source_file"] = file_name  # 원본 파일 이름 저장 (옵션)
        dfs.append(df)
    if dfs:
        folder_concat = pd.concat(dfs, axis=0, ignore_index=True)
        output_file = os.path.join(adjusted_base_path, f"{folder}_concat.csv")
        folder_concat.to_csv(output_file, index=False, encoding='cp949')
        print(f"✅ Saved: {output_file}")
    else:
        print(f"⚠️ No labeled CSV files found in {folder_path}")

In [2]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd

#### 2.) Maker 총합 데이터 기준 Production & Setting 데이터 최빈값 저장
- 추후 추론 시 실시간 데이터 결측보완용 

In [3]:
df_engel = pd.read_csv('임시라벨링_결과/maker1_concat.csv',encoding='cp949')
df_jsw = pd.read_csv('임시라벨링_결과/maker2_concat.csv',encoding='cp949')
df_lsm = pd.read_csv('임시라벨링_결과/maker3_concat.csv',encoding='cp949')

In [9]:
shot_cols = [col for col in df_jsw.columns if col.split('_')[0] == 'shotdata']
mode_dict = df_jsw[shot_cols].mode().iloc[0].to_dict()
mode_df = pd.DataFrame(list(mode_dict.items()), columns=['column', 'mode_value'])
mode_df.to_csv('Mode_values/maker1_sensing_dict.csv', index=False)
mode_df_loaded = pd.read_csv('Mode_values/maker1_sensing_dict.csv')
loaded_mode_dict = dict(zip(mode_df_loaded['column'], mode_df_loaded['mode_value']))

set_cols = [col for col in df_jsw.columns if col.split('_')[0] == 'set']
mode_dict = df_jsw[set_cols].mode().iloc[0].to_dict()
mode_df = pd.DataFrame(list(mode_dict.items()), columns=['column', 'mode_value'])
mode_df.to_csv('Mode_values/maker1_set_dict.csv', index=False)
mode_df_loaded = pd.read_csv('Mode_values/maker1_set_dict.csv')
loaded_mode_dict = dict(zip(mode_df_loaded['column'], mode_df_loaded['mode_value']))

#### 3.) 현재 완성데이터 정보 예시

In [14]:
#### Ex.) 특정 설비 & 품목 & 생산날짜(Lot_No) & Setting변동에 따른 총 생산개수 및 예측 불량률 나열 
group = df_engel.groupby(['MECHCD', 'SABUN', 'lot_no', 'clusterSetting'])['PassOrFail']
df_failrate = pd.DataFrame({'total_count': group.count(),'fail_count': group.sum(),})
df_failrate['fault_rate'] = df_failrate['fail_count'] / df_failrate['total_count']
display(df_failrate.tail(20))

top_total = df_failrate.sort_values('total_count', ascending=False).head(1)
display(top_total)

top_fail = df_failrate.sort_values('fail_count', ascending=False).head(1)
display(top_fail)

total_count  fail_count  \
MECHCD SABUN           lot_no   clusterSetting                            
M1006  W-N1921-37540-A 20250220 30                      380           2   
                       20250305 151                     769           0   
                       20250306 151                    1518           2   
                       20250312 30                      779           0   
                       20250313 30                       47           1   
                       20250316 30                      653           1   
                       20250317 30                      948           2   
                       20250318 30                      978           1   
                       20250319 30                       78           0   
                       20250320 30                      667           0   
                       20250321 30                     1288           3   
                       20250324 30                     2091           4   
                       20250325 30                     1509           0   
                                187                     306           0   
                       20250326 187                     948           0   
                       20250327 187                     978           0   
                       20250328 187                     484           0   
                       20250402 187                     369           0   
                       20250403 187                    1448           3   
                       20250404 187                    1106           0   

                                                fault_rate  
MECHCD SABUN           lot_no   clusterSetting              
M1006  W-N1921-37540-A 20250220 30                0.005263  
                       20250305 151               0.000000  
                       20250306 151               0.001318  
                       20250312 30                0.000000  
                       20250313 30                0.021277  
                       20250316 30                0.001531  
                       20250317 30                0.002110  
                       20250318 30                0.001022  
                       20250319 30                0.000000  
                       20250320 30                0.000000  
                       20250321 30                0.002329  
                       20250324 30                0.001913  
                       20250325 30                0.000000  
                                187               0.000000  
                       20250326 187               0.000000  
                       20250327 187               0.000000  
                       20250328 187               0.000000  
                       20250402 187               0.000000  
                       20250403 187               0.002072  
                       20250404 187               0.000000

,,,,total_count,fail_count,fault_rate
MECHCD,SABUN,lot_no,clusterSetting,,,
1015,W-CG921-37220-A,20240503,8,2673,0,0.0


,,,,total_count,fail_count,fault_rate
MECHCD,SABUN,lot_no,clusterSetting,,,
1015,W-CG921-37220-A,20241202,253,759,153,0.201581


#### 4.) 각 장비(maker)별 고유 Setting Feature 수치 저장

In [21]:
set_cols = [col for col in df_lsm.columns if col.split('_')[0] == 'set']
df_lsm_unique_settings = df_lsm[set_cols + ['clusterSetting']].drop_duplicates().reset_index(drop=True)

set_cols = [col for col in df_jsw.columns if col.split('_')[0] == 'set']
df_jsw_unique_settings = df_jsw[set_cols + ['clusterSetting']].drop_duplicates().reset_index(drop=True)

set_cols = [col for col in df_engel.columns if col.split('_')[0] == 'set']
df_engel_unique_settings = df_engel[set_cols + ['clusterSetting']].drop_duplicates().reset_index(drop=True)

In [23]:
df_lsm_unique_settings.shape, df_jsw_unique_settings.shape, df_engel_unique_settings.shape

((455, 140), (417, 56), (157, 77))

In [25]:
df_engel_unique_settings = np.array(df_engel_unique_settings)
df_jsw_unique_settings = np.array(df_jsw_unique_settings)
df_lsm_unique_settings = np.array(df_lsm_unique_settings)

In [26]:
np.save('setting_values/maker1_settings.npy', df_engel_unique_settings)
np.save('setting_values/maker2_settings.npy', df_jsw_unique_settings)
np.save('setting_values/maker3_settings.npy', df_lsm_unique_settings)